In [7]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : c:\
Directory name is : 


In [1369]:
import os
import glob
import pandas

path = 'C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]
print(result)

['F-179.csv', 'F-319.csv', 'F-45.csv', 'G-1220.csv', 'G-1260_Done.csv', 'G-2147.csv', 'G-2866.csv', 'G-3549.csv', 'G-561.csv', 'G-580A.csv', 'G-852_NO_TIDE.csv', 'G-860.csv', 'PB-1680.csv']


# Read file

In [487]:
# Dont do number 4
# Next to do: g1260, 11 (NO TIDE)

In [488]:
start = '2007-10-01 01:00:00'
end = '2018-06-08 11:00:00'
time_range = pd.date_range(start=start, end=end,freq='H')

In [1372]:
file = result[0]
df = pd.read_csv(file)

#if 'Code' in df:
  # df = df[df.Code == "A"]
file

C:\Users\Christopher\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,4,5,6,7,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'F-179.csv'

In [1373]:
df.Code.describe()

count     111523
unique         1
top            A
freq      111523
Name: Code, dtype: object

In [1242]:
df = df[df.Code != "P"]

In [1243]:
df.Code.unique()

array(['A', nan], dtype=object)

In [1038]:
df[df.RainDateHour == '2018/04/09 11:00']

,TideDateHour,Date,Time,Tide_ft,WellDateHour,date,time,tz_cd,Well_ft,Code,Corrected,RainDateHour,Date.1,RAIN_FT
368972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018/04/09 11:00,4/9/18 11:00,0.0


In [1120]:
df[df.Code == "P"].RAIN_FT.max()

0.018333000000000002

# Aggregate Tide Water Elevation

In [1376]:
#%% Create a set format for which to read the string as a datetime variable

#df['TideDateHour'] = df.loc[:, 'DateHour']
#df['Tide_ft'] = df.loc[:, 'TideLevel']
tide = df.loc[:, ['TideDateHour', 'Tide_ft']]

tide = tide.loc[tide.TideDateHour != '1900/01/00 00:00']

time_format = "%Y/%m/%d %H:%M"

tide['TideDateHour'] = pd.to_datetime(tide.loc[:,'TideDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
tide = tide.set_index('TideDateHour')

#%% Select dates in the range required
data_tide = tide['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_tide = data_tide.resample('H').mean()

In [1245]:
df.head()

,Date,Time,Tide_ft,Corrected,Date.1,Time.1,tz_cd,Well_ft,Code,Corrected.1,Date.2,RAIN_FT
0,NaN,NaN,NaN,0.242,7/28/2000,8:24,EDT,2.26,A,0.702,10/10/2005 0:00,0.0
1,NaN,NaN,NaN,0.262,8/11/2000,10:04,EDT,2.22,A,0.662,10/10/2005 0:15,0.0
2,NaN,NaN,NaN,0.282,10/12/2000,17:18,EDT,3.85,A,2.292,10/10/2005 0:30,0.0
3,NaN,NaN,NaN,0.312,1/8/2001,9:47,EST,1.84,A,0.282,10/10/2005 0:45,0.0
4,NaN,NaN,NaN,0.342,4/25/2001,9:32,EDT,1.51,A,-0.048,10/10/2005 1:00,0.0


In [1377]:
# Aggregate Rain

#%% Create a set format for which to read the string as a datetime variable
rain = df.loc[:, ['RainDateHour', 'RAIN_FT']]

time_format = "%Y/%m/%d %H:%M"

rain['RainDateHour'] = pd.to_datetime(rain['RainDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
rain = rain.set_index('RainDateHour')

#%% Select dates in the range required
data_rain = rain['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_rain = data_rain.resample('H').mean()

# Aggregate Well

#%% Create a set format for which to read the string as a datetime variable
well = df.loc[:, ['WellDateHour', 'Well_ft']]

well = well.loc[well.WellDateHour != '1900/01/00 00:00']
well['WellDateHour'] = pd.to_datetime(well['WellDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
well = well.set_index('WellDateHour')

#%% Select dates in the range required
data_narrow = well['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_well = data_narrow.resample('H').mean()

data_well['Well_ft'] = data_well.Well_ft.interpolate()

# Merge all of the data into one table and save

In [1378]:
file

'F-179.csv'

In [1379]:
len(time_range)

93683

In [1380]:
full = pd.DataFrame(index=pd.DatetimeIndex(time_range))

In [1381]:
full.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 93683 entries, 2007-10-01 01:00:00 to 2018-06-08 11:00:00
Freq: H
Empty DataFrame

In [1382]:
full['Well_ft'] = data_well['Well_ft']
full['Tide_ft'] = data_tide['Tide_ft']
full['Rain_ft'] = data_rain['RAIN_FT']
full.index.name = 'DateHour'

In [1383]:
full.Tide_ft.interpolate(inplace=True)
full.Rain_ft.interpolate(inplace=True)
full.Well_ft.interpolate(inplace=True)

# Fill in empty begininngs and ends
avg_tide = full.Tide_ft.mean()
full.Tide_ft.fillna(value=avg_tide, inplace=True)
full.Tide_ft.fillna(value=avg_tide, inplace=True)

full.Rain_ft.fillna(value=0, inplace=True)
full.Rain_ft.fillna(value=0, inplace=True)

avg_well = full.Well_ft.mean()
full.Well_ft.fillna(value=avg_well, inplace=True)
full.Well_ft.fillna(value=avg_well, inplace=True)


In [1384]:
assert not sum(full.Tide_ft.isna())
assert not sum(full.Rain_ft.isna())
assert not sum(full.Well_ft.isna())

In [1392]:
full.loc['09/10/2017 05:00:00':, 'Rain_ft'].head()

DateHour
2007-10-01 01:00:00    0.001771
2007-10-01 02:00:00    0.002146
2007-10-01 03:00:00    0.005104
2007-10-01 04:00:00    0.004792
2007-10-01 05:00:00    0.002188
Name: Rain_ft, dtype: float64

In [1387]:
new_file = "final_wells/" + file[0:len(file)-4].replace("-", "") + "_ALL.csv"

In [1388]:
new_file

'final_wells/F179_ALL.csv'

In [1389]:
full.to_csv(new_file, index=True)

In [1248]:
file

'G-1260_Done.csv'

In [1249]:
df.to_csv("final_wells/G1260_ALL.csv")

# Incorporate Predicted Well Water Elevation After SAS Prediction

In [1390]:
file = result[0]
well = file[:len(file)-4].replace("_", "").replace("-", "")
well_name = "Well " + well #file[:len(file)-9]
print(well_name)
print(well)

Well F179
F179


In [1256]:
pred = pd.read_sas("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_all_pred.sas7bdat")
full = pd.read_csv("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_ALL.csv", index_col='DateHour')
pred.set_index(full.index, inplace=True)

full[well_name + " Well Level"] = full.Well_ft
full[well_name + " Tide Level"] = full.Tide_ft
full[well_name + " Rain Level"] = full.Rain_ft
full[well_name + " Predicted Well Level"] = pred['FORECAST']
full[well_name + " Lower 95% Confidence Interval"] = pred.L95
full[well_name + " Upper 95% Confidence Interval"] = pred.U95

In [1258]:
full[well_name + " Well Level"] = full.Well_ft
full[well_name + " Tide Level"] = full.Tide_ft
full[well_name + " Rain Level"] = full.Rain_ft
full[well_name + " Predicted Well Level"] = pred['FORECAST']
full[well_name + " Lower 95% Confidence Interval"] = pred.L95
full[well_name + " Upper 95% Confidence Interval"] = pred.U95

In [1391]:
pred = pd.read_sas("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_all_pred.sas7bdat")
full = pd.read_csv("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_ALL.csv", index_col='DateHour')
pred.set_index(full.index, inplace=True)

full[well_name + " Well Level"] = full.Well_ft
full[well_name + " Tide Level"] = full.Tide_ft
full[well_name + " Rain Level"] = full.Rain_ft
full[well_name + " Predicted Well Level"] = pred['FORECAST']
full[well_name + " Lower 95% Confidence Interval"] = pred.L95
full[well_name + " Upper 95% Confidence Interval"] = pred.U95

full.head()

,Well_ft,Tide_ft,Rain_ft,Well F179 Well Level,Well F179 Tide Level,Well F179 Rain Level,Well F179 Predicted Well Level,Well F179 Lower 95% Confidence Interval,Well F179 Upper 95% Confidence Interval
DateHour,,,,,,,,,
2007-10-01 01:00:00,3.1775,0.965,0.001771,3.1775,0.965,0.001771,NaN,NaN,NaN
2007-10-01 02:00:00,3.1900,0.358,0.002146,3.1900,0.358,0.002146,NaN,NaN,NaN
2007-10-01 03:00:00,3.2075,-0.144,0.005104,3.2075,-0.144,0.005104,NaN,NaN,NaN
2007-10-01 04:00:00,3.2275,-0.623,0.004792,3.2275,-0.623,0.004792,NaN,NaN,NaN
2007-10-01 05:00:00,3.2550,-0.994,0.002188,3.2550,-0.994,0.002188,3.288089,3.277932,3.298246


In [1393]:
#full = full.iloc[:, 3:]
full.to_csv("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + str(well) + "_FINAL.csv", index=True)

# Aggregate all data

In [1394]:
import os
import glob
import pandas

path = 'C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]

term = result[1]
print(term)
result = [i for i in result if i[(len(i)-9):len(i)] == "FINAL.csv"]
print(result)

F179_ALL.csv
9_ALL.csv
['F179_FINAL.csv', 'F319_FINAL.csv', 'F45_FINAL.csv', 'G1220_FINAL.csv', 'G1260_FINAL.csv', 'G2147_FINAL.csv', 'G2866_FINAL.csv', 'G3549_FINAL.csv', 'G561_FINAL.csv', 'G580A_FINAL.csv', 'G860_FINAL.csv', 'PB1680_FINAL.csv']


In [1399]:
df = pd.DataFrame(index=time_range)

In [1400]:
well = pd.read_csv(path + result[0])

""
2007-10-01 01:00:00
2007-10-01 02:00:00
2007-10-01 03:00:00
2007-10-01 04:00:00
2007-10-01 05:00:00


In [1401]:
well.head(10)

,DateHour,Well_ft,Tide_ft,Rain_ft,Well F179 Well Level,Well F179 Tide Level,Well F179 Rain Level,Well F179 Predicted Well Level,Well F179 Lower 95% Confidence Interval,Well F179 Upper 95% Confidence Interval
0,2007-10-01 01:00:00,3.1775,0.965,0.001771,3.1775,0.965,0.001771,NaN,NaN,NaN
1,2007-10-01 02:00:00,3.1900,0.358,0.002146,3.1900,0.358,0.002146,NaN,NaN,NaN
2,2007-10-01 03:00:00,3.2075,-0.144,0.005104,3.2075,-0.144,0.005104,NaN,NaN,NaN
3,2007-10-01 04:00:00,3.2275,-0.623,0.004792,3.2275,-0.623,0.004792,NaN,NaN,NaN
4,2007-10-01 05:00:00,3.2550,-0.994,0.002188,3.2550,-0.994,0.002188,3.288089,3.277932,3.298246
5,2007-10-01 06:00:00,3.2850,-1.168,0.004125,3.2850,-1.168,0.004125,3.302733,3.292576,3.312890
6,2007-10-01 07:00:00,3.3100,-0.846,0.003042,3.3100,-0.846,0.003042,3.315988,3.305831,3.326145
7,2007-10-01 08:00:00,3.4550,-0.509,0.001646,3.4550,-0.509,0.001646,3.352260,3.342103,3.362417
8,2007-10-01 09:00:00,3.6500,0.125,0.000104,3.6500,0.125,0.000104,3.530452,3.520295,3.540609
9,2007-10-01 10:00:00,3.7250,0.584,0.002229,3.7250,0.584,0.002229,3.722981,3.712824,3.733138


In [1405]:
df['Well Level'] = well.iloc[:, 0]
df['Tide Level'] = well.iloc[:, 1]
df['Rain Level'] = well.iloc[:, 2]
df['Predicted Well Level'] = well.iloc[:,7]
df['Well Name'] = result[0]

In [1303]:
df = pd.DataFrame(new_df)

In [1406]:
df.head(10)

,Well Level,Tide Level,Rain Level,Predicted Well Level,Well Name
2007-10-01 01:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 02:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 03:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 04:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 05:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 06:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 07:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 08:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 09:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv
2007-10-01 10:00:00,NaN,NaN,NaN,NaN,F179_FINAL.csv


In [1434]:
df.loc[df['DateHour'] == '2017-09-10 12:00:00', ['Rain Level', 'Well Level', 'Predicted Well Level']]

,Rain Level,Well Level,Predicted Well Level
87179,0.007500,2.680000,2.680646
87179,0.010833,4.200000,4.144165
87179,0.004375,3.840000,3.827370
87179,0.006667,2.613372,2.679790
87179,0.022500,6.012000,5.993920
87179,0.005417,6.790000,6.810727
87179,0.005625,8.250435,8.236779
87179,0.018750,3.680000,3.571826
87179,0.006875,4.840000,4.833300
87179,0.011042,3.720000,3.663194


In [1435]:
len(df)/12

93683.0

In [1431]:
i=12
well = pd.read_csv(path + result[i])

new_df = {
    'DateHour': well.iloc[:, 0],
    'Well Level': well.iloc[:, 1],
    'Tide Level': well.iloc[:, 2],
    'Rain Level': well.iloc[:, 3],
    'Predicted Well Level': well.iloc[:,7],
    'Well Name': pd.Series([result[i][:len(result[i])-10] for name in range(len(well))])
}
#print(well.head())
#df = pd.DataFrame(new_df)

df = df.append(pd.DataFrame(new_df))
print(df.tail())
print(len(df))

IndexError: list index out of range

In [1306]:
len(df)

187366

In [1318]:
df['Well Name'].unique()

array(['F179', 'F319', 'F45', 'G1220', 'G1260', 'G2147', 'G2866', 'G3549',
       'G561', 'G580A', 'G860', 'PB1680'], dtype=object)

In [1454]:
0.008819*12

0.105828

In [1453]:
test['2008 July 8 00:00:00']

,DateHour,Predicted Well Level,Rain Level,Tide Level,Well Level,Well Name
DateHour,,,,,,
2008-07-08,2008-07-08 00:00:00,2.409600,0.000000,0.194,2.420,F179
2008-07-08,2008-07-08 00:00:00,2.552621,0.000000,0.194,2.540,F319
2008-07-08,2008-07-08 00:00:00,2.384075,0.000000,-1.212,2.390,F45
2008-07-08,2008-07-08 00:00:00,1.518461,0.000000,0.307,1.520,G1220
2008-07-08,2008-07-08 00:00:00,3.793960,0.105833,-0.976,3.792,G1260
2008-07-08,2008-07-08 00:00:00,4.256672,0.020208,0.126,4.210,G2147
2008-07-08,2008-07-08 00:00:00,6.277313,0.032292,0.126,6.280,G2866
2008-07-08,2008-07-08 00:00:00,2.127667,0.000000,-0.436,2.130,G3549
2008-07-08,2008-07-08 00:00:00,1.952280,0.000000,0.307,1.970,G561


ValueError: cannot reindex from a duplicate axis

In [1452]:
test = df.set_index(pd.DatetimeIndex(df['DateHour']))
test.loc[test['Well Name'] == 'G1260', 'Rain Level'] *= 12

In [1461]:
df.loc[df['Well Name'] == 'G1260', 'Rain Level'] = test.loc[test['Well Name'] == 'G1260', 'Rain Level']

In [1456]:
sum(df[['Rain Level']] == test[['Rain Level']])

ValueError: Can only compare identically-labeled DataFrame objects

93683.0

In [1465]:
df.to_csv(path + "Combined.csv", index=False)

In [1363]:
df.tail(168)

,DateHour,Predicted Well Level,Rain Level,Tide Level,Well Level,Well Name
93515,2018-06-01 12:00:00,2.248987,0.0,-1.937,2.25,PB1680
93516,2018-06-01 13:00:00,2.247563,0.0,-1.937,2.25,PB1680
93517,2018-06-01 14:00:00,2.245858,0.0,-1.937,2.25,PB1680
93518,2018-06-01 15:00:00,2.243967,0.0,-1.937,2.25,PB1680
93519,2018-06-01 16:00:00,2.241922,0.0,-1.937,2.25,PB1680
93520,2018-06-01 17:00:00,2.239782,0.0,-1.937,2.25,PB1680
93521,2018-06-01 18:00:00,2.237588,0.0,-1.937,2.25,PB1680
93522,2018-06-01 19:00:00,2.235364,0.0,-1.937,2.25,PB1680
93523,2018-06-01 20:00:00,2.233122,0.0,-1.937,2.25,PB1680
93524,2018-06-01 21:00:00,2.230873,0.0,-1.937,2.25,PB1680


In [909]:
wells = df['Well Name'].unique()

In [1462]:
wells

array(['F319', 'F45', 'G1220', 'G1260', 'G2147', 'G2866', 'G3549', 'G561',
       'G580A', 'G860', 'PB1680', 'F179'], dtype=object)

In [916]:
df.loc[df['Well Name'] == 'G1260', ['DateHour', 'Rain Level']].mean()

Rain Level    0.000528
dtype: float64

In [1364]:
test = df.set_index(pd.DatetimeIndex(df.DateHour))

In [1436]:
#Get MAPE for each well prediction
# Mean(yi - yhat / yi)
test = test['2018-06-01 12:00:00':'2018-06-08 11:00:00']
test['res'] = abs(test['Well Level'] - test['Predicted Well Level'])/abs(test['Well Level'])

mapes = test.groupby('Well Name')['res'].mean()

In [1438]:
mapes.median()


0.040360426578609374

# Testing for missing rain data 10/24

In [922]:
df.set_index(pd.DatetimeIndex(df['DateHour']), inplace=True)

In [953]:
sub1 = df.loc['11/04/2010 11:00:00':]
sub1 = sub1[sub1['Well Name'] == "G3549"]

In [933]:
sub = pd.read_csv(path + "../G-3549.csv")

C:\Users\Christopher\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,4,5,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [935]:
sub.set_index(pd.DatetimeIndex(sub.RainDateHour), inplace=True)

In [946]:
sub = sub.loc['11/04/2010 11:00:00':'11/04/2017']